## Install libraries

In [31]:
%%capture
!pip install emoji regex pandas unicodedata

In [32]:
import pandas as pd
import re
import regex
import emoji
import unicodedata

In [33]:
# Đọc file CSV
df = pd.read_csv("/kaggle/input/dataset-with-summaries/dataset_with_summaries.csv")

print("=== Head ===")
print(df.head())

print("\n=== Shape ===")
print(df.shape)

print("\n=== Info ===")
print(df.info())

=== Head ===
   id                                               post  \
0   0  bức ảnh ô nhục: vào thời điểm tấm hình này đượ...   
1   1  bức ảnh ô nhục: vào thời điểm tấm hình này đượ...   
2   2  bức ảnh ô nhục: vào thời điểm tấm hình này đượ...   
3   3  bức ảnh ô nhục: vào thời điểm tấm hình này đượ...   
4   4  bức ảnh ô nhục: vào thời điểm tấm hình này đượ...   

                                             summary  \
0  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
1  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
2  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
3  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
4  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   

                                             comment            label  \
0  luận điệu của bọn phản động, sỏ lá, 3/// viết ...  KHONG_PHAN_DONG   
1  vậy ông bảo đại, ông diệm, ông thiệu là đảng v...  KHONG_PHAN_DONG   
2                              muôn đời của đám 3///  KHONG_PHAN_DONG 

In [34]:
selected_df = df[['summary', 'comment', 'label']]
print(selected_df.head())

print("\n=== Shape ===")
print(selected_df.shape)

print("\n=== Info ===")
print(selected_df.info())

                                             summary  \
0  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
1  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
2  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
3  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
4  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   

                                             comment            label  
0  luận điệu của bọn phản động, sỏ lá, 3/// viết ...  KHONG_PHAN_DONG  
1  vậy ông bảo đại, ông diệm, ông thiệu là đảng v...  KHONG_PHAN_DONG  
2                              muôn đời của đám 3///  KHONG_PHAN_DONG  
3  già rồi mà đần vậy cháu ? cộng sản đánh mỹ, đá...  KHONG_PHAN_DONG  
4  đúng là 3/// xỏ lá, bác hồ mất nên các bác khó...  KHONG_PHAN_DONG  

=== Shape ===
(11827, 3)

=== Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11827 entries, 0 to 11826
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summ

## 1.Normalize Unicode (NFC) + lowercase

In [35]:
import unicodedata
def normalize_unicode_lower(text):
    text = unicodedata.normalize('NFC', text)
    return text.lower()

## 2.Remove Emoji, links/HTML/mentions/hashtags/UI indicators

In [36]:
import re
import emoji
import unicodedata

# Cải thiện các pattern emoticon từ minimal_cleaning.py
EMOTICON_PATTERNS = [
    r":\)+",       # :), :)), :))), ...
    r":\(+",       # :(, :((, ...
    r":v+",        # :v, :vvv, ...
    r":V+",        # :V, :VV, ... (thêm từ minimal_cleaning.py)
    r"=+\)+",      # =), =)), ...
    r"=+\(+",      # =(, =((, ...
    r":d+",        # :d, :dd
    r":p+",        # :p, :pp
    r"<3+",        # <3<3<3
    r"=+\]+",      # =], =]], =]]], ...
    r"=+\[+",      # =[, =[[, =[[[ ...
    r":>+",        # :>, :>>, ... (thêm từ minimal_cleaning.py)
    r":<+",        # :<, :<<, ... (thêm từ minimal_cleaning.py)
    r":\(\(",      # :(( (thêm từ minimal_cleaning.py)
    r"=\(\(",      # =(( (thêm từ minimal_cleaning.py)
]

EMOTICON_REGEX = re.compile("|".join(EMOTICON_PATTERNS), re.IGNORECASE)

# Thêm EMOJI_REGEX từ minimal_cleaning.py để trường hợp không có thư viện emoji
EMOJI_REGEX = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F700-\U0001F77F"  # alchemical symbols
    u"\U0001F780-\U0001F7FF"  # Geometric Shapes
    u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA00-\U0001FA6F"  # Chess Symbols
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251" 
    "]+", flags=re.UNICODE
)

def remove_emoji_emoticon(text):
    """
    Xóa emoji và emoticon từ văn bản.
    Sử dụng thư viện emoji hoặc fallback sang regex nếu thư viện không hoạt động.
    """
    if not isinstance(text, str):
        return ""
    
    # Xóa emoji bằng thư viện emoji
    try:
        text = emoji.replace_emoji(text, replace=" ")
    except:
        # Fallback nếu thư viện emoji gặp lỗi
        text = EMOJI_REGEX.sub(" ", text)
    
    # Xóa emoticon bằng regex
    text = EMOTICON_REGEX.sub(" ", text)
    
    # Loại bỏ khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Regex
URL_REGEX = re.compile(r'https?://\S+|www\.\S+|\S+\.(com|org|net|co|vn|io)(/\S*)?')
HTML_REGEX = re.compile(r"<[^>]+>")
MENTION_REGEX = re.compile(r"@[\w\._]+")  # Cải thiện để bắt dấu . và _
HASHTAG_REGEX = re.compile(r"#\w+")
UI_INDICATORS = [
    "đã chỉnh sửa", "[đã chỉnh sửa]", "(đã chỉnh sửa)",
    "see more", "xem thêm", "see translation", "xem bản dịch",
    "ẩn bớt", "xem ít hơn", "dịch", "translated", "more", "less",
    "see more reactions"
]

def remove_html_url_mention_hashtag(text):
    """
    Loại bỏ URL, HTML tags, mentions, hashtags và các chỉ báo giao diện.
    """
    if not isinstance(text, str):
        return ""
    
    # Chuẩn hóa Unicode (từ minimal_cleaning.py)
    text = unicodedata.normalize('NFC', text)
    
    # Xóa URL
    text = URL_REGEX.sub(" ", text)
    
    # Xóa HTML tags
    text = HTML_REGEX.sub(" ", text)
    
    # Xóa mentions và hashtags
    text = MENTION_REGEX.sub(" ", text)
    text = HASHTAG_REGEX.sub(" ", text)
    
    # Xóa các chỉ báo giao diện (case insensitive)
    for ind in UI_INDICATORS:
        text = re.sub(r'(?i)' + re.escape(ind), " ", text)
    
    # Loại bỏ dấu câu riêng lẻ
    text = re.sub(r'(?<!\w)[\^\'\`\~\"\,\.]+(?!\w)', ' ', text)
    
    # Làm sạch khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

## 3.Remove punctuation + strip extra spaces

In [37]:
ALLOWED_SPECIAL_CHARS = r"+/\|=&\.'|"
VIET_CHARACTERS = (
    r"àáạảãâầấậẩẫăằắặẳẵ"
    r"èéẹẻẽêềếệểễ"
    r"ìíịỉĩ"
    r"òóọỏõôồốộổỗơờớợởỡ"
    r"ùúụủũưừứựửữ"
    r"ỳýỵỷỹ"
    r"đ"
)
INVALID_CHAR_REGEX = regex.compile(
    f"[^{VIET_CHARACTERS}0-9a-zA-Z {ALLOWED_SPECIAL_CHARS}]+",
    flags=regex.IGNORECASE
)

def remove_punctuation_keep_vietnamese(text):
    if not isinstance(text, str):
        return ""
    return INVALID_CHAR_REGEX.sub(" ", text)

def strip_extra_spaces(text):
    if not isinstance(text, str):
        return ""
    return regex.sub(r"\s+", " ", text).strip()

## 4.Reduce elongated characters

In [38]:
import regex

def reduce_elongated(text):
    if not isinstance(text, str):
        return ""
    # \p{L} khớp mọi ký tự chữ Unicode (bao gồm cả tiếng Việt có dấu)
    pattern = regex.compile(r"([\p{L}])\1{2,}", flags=regex.IGNORECASE)
    return pattern.sub(r"\1\1", text)

## 5. De-teencode, Decode Abbreviate 

In [39]:
import re
import uuid
import json

# Load dictionary với error handling
try:
    with open('/kaggle/input/dictionary/text_normalization_map.json', encoding='utf-8') as f:
        norm_dict = json.load(f)
    print(f"✅ Đã load dictionary với {len(norm_dict)} entries")
except FileNotFoundError:
    print("❌ Không tìm thấy file dictionary")
    norm_dict = {}
except Exception as e:
    print(f"❌ Lỗi khi load dictionary: {str(e)}")
    norm_dict = {}

def normalize_teencode(text, dictionary):
    if not isinstance(text, str):
        return ""
    
    # Phân tách các entry thành nhóm "toàn ký tự đặc biệt" và "có chữ/số"
    word_entries = []
    symbol_entries = []
    
    for slang, standard in dictionary.items():
        if re.fullmatch(r"[^\w\s]+", slang):  # chỉ toàn ký tự không phải chữ hoặc số
            symbol_entries.append((slang, standard))
        else:
            word_entries.append((slang, standard))

    # Sắp xếp giảm dần theo độ dài key, để ưu tiên match multi-word trước
    all_entries = sorted(word_entries + symbol_entries, key=lambda x: -len(x[0]))
    temp_map = {}

    for slang, standard in all_entries:
        # Tạo token tạm (unique) để tránh chen chúc khi replace
        temp_token = f"__TMP_{uuid.uuid4().hex}__"
        
        # Nếu slang chỉ toàn ký tự đặc biệt, dùng pattern re.escape; ngược lại, dùng \b...\b
        if re.fullmatch(r"[^\w\s]+", slang):
            pattern = re.compile(re.escape(slang), flags=re.IGNORECASE)
        else:
            # \b đảm bảo slang là một từ độc lập (ví dụ: "ba que" vs "baque")
            pattern = re.compile(rf"\b{re.escape(slang)}\b", flags=re.IGNORECASE)

        # pattern.subn trả về (new_text, số lần replace)
        new_text, count = pattern.subn(temp_token, text)
        if count > 0:
            temp_map[temp_token] = standard
            text = new_text  # cập nhật text ngay để các entry ngắn hơn không đè lên

    # Cuối cùng, thay token tạm bằng từ chuẩn
    for token, standard in temp_map.items():
        text = text.replace(token, standard)

    return text

✅ Đã load dictionary với 462 entries


## 6. Deduplication 

In [40]:
def deduplicate_comments(df, col):
    before = len(df)
    df_nodup = df.drop_duplicates(subset=[col]).reset_index(drop=True)
    after = len(df_nodup)
    return df_nodup

In [41]:
selected_df = selected_df.copy()

# 1. Normalize Unicode (NFC) + lowercase
selected_df['comment_clean'] = selected_df['comment'].apply(normalize_unicode_lower)

# 2. Loại bỏ emoji, HTML, URL, mentions, hashtags, UI indicators
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_html_url_mention_hashtag)

selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_emoji_emoticon)

# 3. Xóa ký tự không phải tiếng Việt/tiếng Anh + làm sạch khoảng trắng
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_punctuation_keep_vietnamese)
selected_df['comment_clean'] = selected_df['comment_clean'].apply(strip_extra_spaces)

# 4. Giảm lặp ký tự kéo dài
selected_df['comment_clean'] = selected_df['comment_clean'].apply(reduce_elongated)

# 5. De-teencode (multi-word nối bằng "_")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(lambda x: normalize_teencode(x, norm_dict))

# 6. Loại bỏ dòng trùng lặp dựa trên comment_clean
selected_df = deduplicate_comments(selected_df, col='comment_clean')

In [42]:
# 8. Xuất file như bình thường
export_df = selected_df[['summary', 'comment_clean', 'label']].copy()
export_df.to_csv("/kaggle/working/final_full_output.csv", index=False, encoding="utf-8")

print("✅ Đã lưu dữ liệu rút gọn vào: final_full_output.csv")
print("\n=== 5 dòng đầu (RÚT GỌN) ===")
print(export_df.head(5))

✅ Đã lưu dữ liệu rút gọn vào: final_full_output.csv

=== 5 dòng đầu (RÚT GỌN) ===
                                             summary  \
0  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
1  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
2  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
3  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
4  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   

                                       comment_clean            label  
0  luận điệu của bọn phản động xỏ lá 3ba que viết...  KHONG_PHAN_DONG  
1  vậy ông bảo đại ông diệm ông thiệu là đảng viê...  KHONG_PHAN_DONG  
2                           muôn đời của đám 3ba que  KHONG_PHAN_DONG  
3  già rồi mà đần vậy cháu cộng sản đánh mỹ đánh ...  KHONG_PHAN_DONG  
4  đúng là 3ba que xỏ lá bác hồ mất nên các bác k...  KHONG_PHAN_DONG  
